In [1]:
import os
import matplotlib.pyplot as plt
import numpy
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model

In [3]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import backend as K

def cosine_decay_with_warmup(global_step,
                             learning_rate_base,
                             total_steps,
                             warmup_learning_rate=0.0,
                             warmup_steps=0,
                             hold_base_rate_steps=0):
    """Cosine decay schedule with warm up period.
    Cosine annealing learning rate as described in:
      Loshchilov and Hutter, SGDR: Stochastic Gradient Descent with Warm Restarts.
      ICLR 2017. https://arxiv.org/abs/1608.03983
    In this schedule, the learning rate grows linearly from warmup_learning_rate
    to learning_rate_base for warmup_steps, then transitions to a cosine decay
    schedule.
    Arguments:
        global_step {int} -- global step.
        learning_rate_base {float} -- base learning rate.
        total_steps {int} -- total number of training steps.
    Keyword Arguments:
        warmup_learning_rate {float} -- initial learning rate for warm up. (default: {0.0})
        warmup_steps {int} -- number of warmup steps. (default: {0})
        hold_base_rate_steps {int} -- Optional number of steps to hold base learning rate
                                    before decaying. (default: {0})
    Returns:
      a float representing learning rate.
    Raises:
      ValueError: if warmup_learning_rate is larger than learning_rate_base,
        or if warmup_steps is larger than total_steps.
    """

    if total_steps < warmup_steps:
        raise ValueError('total_steps must be larger or equal to '
                         'warmup_steps.')
    learning_rate = 0.5 * learning_rate_base * (1 + np.cos(
        np.pi *
        (global_step - warmup_steps - hold_base_rate_steps
         ) / float(total_steps - warmup_steps - hold_base_rate_steps)))
    if hold_base_rate_steps > 0:
        learning_rate = np.where(global_step > warmup_steps + hold_base_rate_steps,
                                 learning_rate, learning_rate_base)
    if warmup_steps > 0:
        if learning_rate_base < warmup_learning_rate:
            raise ValueError('learning_rate_base must be larger or equal to '
                             'warmup_learning_rate.')
        slope = (learning_rate_base - warmup_learning_rate) / warmup_steps
        warmup_rate = slope * global_step + warmup_learning_rate
        learning_rate = np.where(global_step < warmup_steps, warmup_rate,
                                 learning_rate)
    return np.where(global_step > total_steps, 0.0, learning_rate)


class WarmUpCosineDecayScheduler(keras.callbacks.Callback):
    """Cosine decay with warmup learning rate scheduler
    """

    def __init__(self,
                 learning_rate_base,
                 total_steps,
                 global_step_init=0,
                 warmup_learning_rate=0.0,
                 warmup_steps=0,
                 hold_base_rate_steps=0,
                 verbose=0):
        """Constructor for cosine decay with warmup learning rate scheduler.
    Arguments:
        learning_rate_base {float} -- base learning rate.
        total_steps {int} -- total number of training steps.
    Keyword Arguments:
        global_step_init {int} -- initial global step, e.g. from previous checkpoint.
        warmup_learning_rate {float} -- initial learning rate for warm up. (default: {0.0})
        warmup_steps {int} -- number of warmup steps. (default: {0})
        hold_base_rate_steps {int} -- Optional number of steps to hold base learning rate
                                    before decaying. (default: {0})
        verbose {int} -- 0: quiet, 1: update messages. (default: {0})
        """

        super(WarmUpCosineDecayScheduler, self).__init__()
        self.learning_rate_base = learning_rate_base
        self.total_steps = total_steps
        self.global_step = global_step_init
        self.warmup_learning_rate = warmup_learning_rate
        self.warmup_steps = warmup_steps
        self.hold_base_rate_steps = hold_base_rate_steps
        self.verbose = verbose
        self.learning_rates = []

    def on_batch_end(self, batch, logs=None):
        self.global_step = self.global_step + 1
        lr = K.get_value(self.model.optimizer.lr)
        self.learning_rates.append(lr)

    def on_batch_begin(self, batch, logs=None):
        lr = cosine_decay_with_warmup(global_step=self.global_step,
                                      learning_rate_base=self.learning_rate_base,
                                      total_steps=self.total_steps,
                                      warmup_learning_rate=self.warmup_learning_rate,
                                      warmup_steps=self.warmup_steps,
                                      hold_base_rate_steps=self.hold_base_rate_steps)
        K.set_value(self.model.optimizer.lr, lr)
        if self.verbose > 0:
            print('\nBatch %05d: setting learning '
                  'rate to %s.' % (self.global_step + 1, lr))

In [4]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred = K.cast(y_pred, 'float32')
    y_pred_f = K.cast(K.greater(K.flatten(y_pred), 0.5), 'float32')
    intersection = y_true_f * y_pred_f
    score = 2. * K.sum(intersection) / (K.sum(y_true_f) + K.sum(y_pred_f))
    return score


def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score


def bce_dice_loss(y_true, y_pred):
    return K.binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)


def bce_logdice_loss(y_true, y_pred):
    return K.binary_crossentropy(y_true, y_pred) - K.log(1. - dice_loss(y_true, y_pred))


def euclidean_distance_loss(y_true, y_pred):
    """
    Euclidean distance loss
    https://en.wikipedia.org/wiki/Euclidean_distance
    :param y_true: TensorFlow/Theano tensor
    :param y_pred: TensorFlow/Theano tensor of the same shape as y_true
    :return: float
    """
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))


def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy

    Variables:
        weights: numpy array of shape (C,) where C is the number of classes

    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """

    weights = K.variable(weights)

    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss

    return loss


def binary_focal_loss(gamma=2., alpha=.25):
    """
    Binary form of focal loss.
      FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)
      where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.
    References:
        https://arxiv.org/pdf/1708.02002.pdf
    Usage:
     model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def binary_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        epsilon = K.epsilon()
        # clip to prevent NaN's and Inf's
        pt_1 = K.clip(pt_1, epsilon, 1. - epsilon)
        pt_0 = K.clip(pt_0, epsilon, 1. - epsilon)

        return -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) \
               -K.mean((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))

    return binary_focal_loss_fixed


def categorical_focal_loss(gamma=2., alpha=.25):
    """
    Softmax version of focal loss.
           m
      FL = ∑  -alpha * (1 - p_o,c)^gamma * y_o,c * log(p_o,c)
          c=1
      where m = number of classes, c = class and o = observation
    Parameters:
      alpha -- the same as weighing factor in balanced cross entropy
      gamma -- focusing parameter for modulating factor (1-p)
    Default value:
      gamma -- 2.0 as mentioned in the paper
      alpha -- 0.25 as mentioned in the paper
    References:
        Official paper: https://arxiv.org/pdf/1708.02002.pdf
        https://www.tensorflow.org/api_docs/python/tf/keras/backend/categorical_crossentropy
    Usage:
     model.compile(loss=[categorical_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def categorical_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred: A tensor resulting from a softmax
        :return: Output tensor.
        """

        # Scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)

        # Clip the prediction value to prevent NaN's and Inf's
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

        # Calculate Cross Entropy
        cross_entropy = -y_true * K.log(y_pred)

        # Calculate Focal Loss
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy

        # Compute mean loss in mini_batch
        return K.mean(loss, axis=1)

    return categorical_focal_loss_fixed

In [5]:
def recall(y_target, y_pred):
    """ Compare the predicted value and target value and find the true positive and true positive + false negative.
    Then Calculate the recall value using those values.

        Args:
            y_target (list<int>): list of the correct class of the image
            y_pred (list<float>): list of possibility of the class in percentage
        Returns:
            recall (float): recall value

        Example:
            > > > recall([0,1,1,0,1,1,0],[0,1,0,1,1,0,0])

    """
    y_target_yn = K.round(K.clip(y_target, 0, 1))
    y_pred_yn = K.round(K.clip(y_pred, 0, 1))

    # True Positive is when target and prediciton is both positive
    count_true_positive = K.sum(y_target_yn * y_pred_yn)

    # (True Positive + False Negative) = true is 1
    count_true_positive_false_negative = K.sum(y_target_yn)

    # Recall =  (True Positive) / (True Positive + False Negative)
    recall_val = count_true_positive / (count_true_positive_false_negative + K.epsilon())

    # return a single tensor value
    return recall_val


def precision(y_target, y_pred):
    """ Compare the predicted value and target value and find the true positive and true positive + false positive.
        Then Calculate the precision value using those values.

        Args:
            y_target (list<int>): list of the correct class of the image
            y_pred (list<float>): list of possibility of the class in percentage
        Returns:
            precision (float): precision value

        Example:
            > > > precision([0,1,1,0,1,1,0],[0,1,0,1,1,0,0])

    """
    y_pred_yn = K.round(K.clip(y_pred, 0, 1))
    y_target_yn = K.round(K.clip(y_target, 0, 1))

    # True Positive is when target and prediciton is both positive
    count_true_positive = K.sum(y_target_yn * y_pred_yn)

    # (True Positive + False Positive) = prediction is 1
    count_true_positive_false_positive = K.sum(y_pred_yn)

    # Precision = (True Positive) / (True Positive + False Positive)
    precision_val = count_true_positive / (count_true_positive_false_positive + K.epsilon())

    # return a single tensor value
    return precision_val


def sensitivity(y_target, y_pred):
    """ Compare the predicted value and target value and find the true positive and true positive + false negative.
        Then Calculate the sensitivity value using those values.

        Args:
            y_target (list<int>): list of the correct class of the image
            y_pred (list<float>): list of possibility of the class in percentage
        Returns:
            sensitivity (float): sensitivity value

        Example:
            > > > sensitivity([0,1,1,0,1,1,0],[0,1,0,1,1,0,0])

    """
    true_positives = K.sum(K.round(K.clip(y_target * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_target, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())


def specificity(y_target, y_pred):
    """ Compare the predicted value and target value and find the true negative and true negative + false positive.
        Then Calculate the speicificity value using those values.

        Args:
            y_target (list<int>): list of the correct class of the image
            y_pred (list<float>): list of possibility of the class in percentage
        Returns:
            specificity (float): specificity value

        Example:
            > > > specificity([0,1,1,0,1,1,0],[0,1,0,1,1,0,0])

    """
    true_negatives = K.sum(K.round(K.clip((1-y_target) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_target, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())


def f1score(y_target, y_pred):
    """ Use the y_target and y_pred to calculate the recall and precision values.
    Then calculate the f1-score using the recall and precision values.

        Args:
            y_target (list<int>): list of the correct class of the image
            y_pred (list<float>): list of possibility of the class in percentage
        Returns:
            f1score (float): f1score value

        Example:
            > > > f1score([0,1,1,0,1,1,0],[0,1,0,1,1,0,0])

    """
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    _f1score = (2 * _recall * _precision) / (_recall + _precision + K.epsilon())

    # return a single tensor value
    return _f1score


def mean_iou(y_true, y_pred):
    y_pred = K.cast(K.greater(y_pred, .5), dtype='float32') # .5 is the threshold
    inter = K.sum(K.sum(K.squeeze(y_true * y_pred, axis=3), axis=2), axis=1)
    union = K.sum(K.sum(K.squeeze(y_true + y_pred, axis=3), axis=2), axis=1) - inter
    return K.mean((inter + K.epsilon()) / (union + K.epsilon()))

In [12]:
import numpy as np
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from tensorflow.keras.models import *
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *

from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
# from ai_inference.models.filter_response_normalization import FilterResponseNormalization


def unet_gauss(pretrained_weights=None, input_size=(256, 256, 1), dropout_1=0.5, dropout_2=0.5, noise=False):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation=None, padding='same', kernel_initializer='he_normal', name='conv2d_1')(inputs)
    conv1 = BatchNormalization()(conv1)
    if noise:
        conv1 = GaussianNoise(0.1)(conv1)
    conv1 = ReLU()(conv1)
    conv1 = Conv2D(64, 3, activation=None, padding='same', kernel_initializer='he_normal', name='conv2d_2')(conv1)
    conv1 = BatchNormalization()(conv1)
    if noise:
        conv1 = GaussianNoise(0.1)(conv1)
    conv1 = ReLU()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation=None, padding='same', kernel_initializer='he_normal', name='conv2d_3')(pool1)
    conv2 = BatchNormalization()(conv2)
    if noise:
        conv2 = GaussianNoise(0.1)(conv2)
    conv2 = ReLU()(conv2)
    conv2 = Conv2D(128, 3, activation=None, padding='same', kernel_initializer='he_normal', name='conv2d_4')(conv2)
    conv2 = BatchNormalization()(conv2)
    if noise:
        conv2 = GaussianNoise(0.1)(conv2)
    conv2 = ReLU()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation=None, padding='same', kernel_initializer='he_normal', name='conv2d_5')(pool2)
    conv3 = BatchNormalization()(conv3)
    if noise:
        conv3 = GaussianNoise(0.1)(conv3)
    conv3 = ReLU()(conv3)
    conv3 = Conv2D(256, 3, activation=None, padding='same', kernel_initializer='he_normal', name='conv2d_6')(conv3)
    conv3 = BatchNormalization()(conv3)
    if noise:
        conv3 = GaussianNoise(0.1)(conv3)
    conv3 = ReLU()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation=None, padding='same', kernel_initializer='he_normal', name='conv2d_7')(pool3)
    conv4 = BatchNormalization()(conv4)
    if noise:
        conv4 = GaussianNoise(0.1)(conv4)
    conv4 = ReLU()(conv4)
    conv4 = Conv2D(512, 3, activation=None, padding='same', kernel_initializer='he_normal', name='conv2d_8')(conv4)
    conv4 = BatchNormalization()(conv4)
    if noise:
        conv4 = GaussianNoise(0.1)(conv4)
    conv4 = ReLU()(conv4)
    drop4 = GaussianDropout(dropout_1)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation=None, padding='same', kernel_initializer='he_normal', name='conv2d_9')(pool4)
    conv5 = BatchNormalization()(conv5)
    if noise:
        conv5 = GaussianNoise(0.1)(conv5)
    conv5 = ReLU()(conv5)
    conv5 = Conv2D(1024, 3, activation=None, padding='same', kernel_initializer='he_normal', name='conv2d_10')(conv5)
    conv5 = BatchNormalization()(conv5)
    if noise:
        conv5 = GaussianNoise(0.1)(conv5)
    conv5 = ReLU()(conv5)
    drop5 = GaussianDropout(dropout_2)(conv5)

    up6 = Conv2D(512, 2, activation=None, padding='same', kernel_initializer='he_normal', name='conv2d_11')(
        UpSampling2D(size=(2, 2))(drop5))
    merge6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(512, 3, activation=None, padding='same', kernel_initializer='he_normal', name='conv2d_12')(merge6)
    conv6 = BatchNormalization()(conv6)
    if noise:
        conv6 = GaussianNoise(0.1)(conv6)
    conv6 = ReLU()(conv6)
    conv6 = Conv2D(512, 3, activation=None, padding='same', kernel_initializer='he_normal', name='conv2d_13')(conv6)
    conv6 = BatchNormalization()(conv6)
    if noise:
        conv6 = GaussianNoise(0.1)(conv6)
    conv6 = ReLU()(conv6)

    up7 = Conv2D(256, 2, activation=None, padding='same', kernel_initializer='he_normal', name='conv2d_14')(
        UpSampling2D(size=(2, 2))(conv6))
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3, activation=None, padding='same', kernel_initializer='he_normal', name='conv2d_15')(merge7)
    conv7 = BatchNormalization()(conv7)
    if noise:
        conv7 = GaussianNoise(0.1)(conv7)
    conv7 = ReLU()(conv7)
    conv7 = Conv2D(256, 3, activation=None, padding='same', kernel_initializer='he_normal', name='conv2d_16')(conv7)
    conv7 = BatchNormalization()(conv7)
    if noise:
        conv7 = GaussianNoise(0.1)(conv7)
    conv7 = ReLU()(conv7)

    up8 = Conv2D(128, 2, activation=None, padding='same', kernel_initializer='he_normal', name='conv2d_17')(
        UpSampling2D(size=(2, 2))(conv7))
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3, activation=None, padding='same', kernel_initializer='he_normal', name='conv2d_18')(merge8)
    conv8 = BatchNormalization()(conv8)
    if noise:
        conv8 = GaussianNoise(0.1)(conv8)
    conv8 = ReLU()(conv8)
    conv8 = Conv2D(128, 3, activation=None, padding='same', kernel_initializer='he_normal', name='conv2d_19')(conv8)
    conv8 = BatchNormalization()(conv8)
    if noise:
        conv8 = GaussianNoise(0.1)(conv8)
    conv8 = ReLU()(conv8)

    up9 = Conv2D(64, 2, activation=None, padding='same', kernel_initializer='he_normal', name='conv2d_20')(
        UpSampling2D(size=(2, 2))(conv8))
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(64, 3, activation=None, padding='same', kernel_initializer='he_normal', name='conv2d_21')(merge9)
    conv9 = BatchNormalization()(conv9)
    if noise:
        conv9 = GaussianNoise(0.1)(conv9)
    conv9 = ReLU()(conv9)
    conv9 = Conv2D(64, 3, activation=None, padding='same', kernel_initializer='he_normal', name='conv2d_22')(conv9)
    conv9 = BatchNormalization()(conv9)
    if noise:
        conv9 = GaussianNoise(0.1)(conv9)
    conv9 = ReLU()(conv9)
    conv9 = Conv2D(2, 3, activation=None, padding='same', kernel_initializer='he_normal', name='conv2d_23')(conv9)
    conv9 = BatchNormalization()(conv9)
    conv9 = ReLU()(conv9)
    conv10 = Conv2D(1, 1, activation='sigmoid', name='conv2d_50')(conv9)


    model = Model(inputs=inputs, outputs=conv10)

    # model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

    # model.summary()

    if (pretrained_weights):
        model.load_weights(pretrained_weights)

    return model

Using TensorFlow backend.


In [6]:
import os
from os.path import join
import numpy as np

#####################################
# Training Model Callback Parameters
# ModelCheckpoint
MODEL_EXTENSION = '.h5'
MONITOR = 'val_loss'
VERBOSE = 1
SAVE_BEST_ONLY = True
SAVE_WEIGHTS_ONLY = False
MODE = 'min'

# ReduceLROnPlateau
FACTOR = np.sqrt(0.5)
COOLDOWN = 0
LR_PATIENCE = 8
MIN_LR = 0.005e-20

# EarlyStopping
MIN_DELTA = 0.001
ES_PATIENCE = 1000

# CSVLOGGER
CSV_EXTENSION = '.csv'
#####################################

SCOL_WEIGHT_DIR = 'C:/Users/sungjoo/Deepnoid/cf_weight/'
SCOL_CSV_DIR = 'C:/Users/sungjoo/Deepnoid/cf_csv/'
BASE_DIR_SCOL_V1 = 'C:/Users/sungjoo/Desktop/Compression_V1/'
# BASE_DIR_SCOL_V2 = '/data/hira2020/scoliosis/SungJoo/scoliosis_v2/'
# BASE_DIR_SCOL_V3 = '/data/hira2020/scoliosis/SungJoo/scoliosis_v3/'
# BASE_DIR_SCOL_V4 = '/data/hira2020/scoliosis/SungJoo/scoliosis_v4/'

# CF_SPINE_WEIGHT_DIR = 'C:/Users/sungjoo/Deepnoid/cf_weight/'
# CF_SPINE_CSV_DIR = 'C:/Users/sungjoo/Deepnoid/cf_csv/'
# BASE_DIR_CF_SPINE_V1 = 'C:/Users/sungjoo/Desktop/Compression_V1/'
# BASE_DIR_CF_SPINE_V2 = 'C:/Users/sungjoo/Desktop/Compression_V1/'
# BASE_DIR_CF_SPINE_V3 = 'C:/Users/sungjoo/Desktop/Compression_V1/'

In [11]:
!pip install scikit-image

In [23]:
import math
# from unet_bn_gauss import unet_gauss
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping, ModelCheckpoint
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# from cosine_lr_reducer import WarmUpCosineDecayScheduler
# from losses import bce_logdice_loss, dice_loss, euclidean_distance_loss, dice_coef
# from evaluation_metrics import sensitivity, specificity, f1score, recall, mean_iou
# from config import *


def set_gpus():
    physical_devices = tf.config.list_physical_devices('GPU')
    print("Physical_devices")
    print(physical_devices)
    try:
        for pd in physical_devices:
            tf.config.experimental.set_memory_growth(pd, True)
    except:
        pass


def get_callbacks(file_name, cosine_lr, lr, save_best_only=False):
    """ Getter function for the callbacks that will be used while training keras model. In current version,
    model saver when best performance, learning rate reducer and early stopper when no increase in performance spotted,
    and csv logger for keeping track of all the performance for each epoch.
    """

    # Initiating Callback objects
    if save_best_only:
        file_path = SCOL_WEIGHT_DIR + file_name + MODEL_EXTENSION
        checkpoint = ModelCheckpoint(file_path, monitor='val_dice_coef', verbose=VERBOSE, save_best_only=SAVE_BEST_ONLY,
                                     save_weights_only=SAVE_WEIGHTS_ONLY, mode='max', period=1)
    else:
        file_path = SCOL_WEIGHT_DIR + file_name + '-{epoch:02d}' + MODEL_EXTENSION
        checkpoint = ModelCheckpoint(file_path, monitor=MONITOR, verbose=VERBOSE, save_best_only=SAVE_BEST_ONLY,
                                     save_weights_only=SAVE_WEIGHTS_ONLY, mode=MODE, period=1)

    if cosine_lr:
        lr_reducer = WarmUpCosineDecayScheduler(learning_rate_base=lr,
                                            total_steps=int(300*1039/1),
                                            warmup_learning_rate=0.0,
                                            warmup_steps=(10*1039)/1,
                                            hold_base_rate_steps=0)
    else:
        lr_reducer = ReduceLROnPlateau(factor=FACTOR, cooldown=COOLDOWN, patience=LR_PATIENCE, min_lr=MIN_LR)

    early_stopper = EarlyStopping(min_delta=MIN_DELTA, patience=ES_PATIENCE)
    csv_logger = CSVLogger(SCOL_CSV_DIR + file_name + CSV_EXTENSION)

    return [checkpoint, lr_reducer, early_stopper, csv_logger]


def train_scoliosis(batch_size, loss_name, lr=0.01, width=1024, height=1024, norm_func='bn', do1=0.5, do2=0.5, cosine_lr=False, aug_funcs=[], spec_name='', epoch=500):
    # set_gpus('4, 6, 7')
    """"""
    # LOSS EXPERIMENTS
    if loss_name == 'bce_logdice_loss':
        loss = bce_logdice_loss
    elif loss_name == 'dice_loss':
        loss = dice_loss
    elif loss_name == 'l2_loss':
        loss = euclidean_distance_loss
    else:
        loss = loss_name

    if cosine_lr:
        file_name = 'cf_train_no_bin_bckgnd_resized_unet_' + norm_func + 'resize_' + str(loss_name) + '_' + str(
            width) +str(height)+ '_bs-' + str(batch_size) + '_adam-cosine_lr-' + str(lr) + '_do1_' + str(do1) + '_do2_' + str(
            do2) + '_aug_added_fctr_0.5'
    else:
        file_name = 'cf_train_no_bin_bckgnd_resized_unet_'+norm_func+'resize_'+str(loss_name)+'_'+str(width)+str(height)+\
                '_bs-'+str(batch_size)+'_adam-'+str(lr)+'_do1_'+str(do1)+'_do2_'+str(do2)+'_aug_added_fctr_0.5'
    if spec_name != '':
        file_name += '_'+spec_name
    # file_name = 'mmg_ca_train_median_gaussian_clahe_unet_resize_bce_1024_bs-1_v4.1'
    # file_name = 'mmg_ca_train_median_gaussian_clahe_unet_resize_dice_1024_bs-1_v4.1'
    """"""
    callbacks = get_callbacks(file_name, cosine_lr, lr)
    set_gpus()
    # tf.config.experimental.set_memory_growth(device, enable)
    strategy = tf.distribute.MirroredStrategy()
    # cross_device_ops = tf.distribute.HierarchicalCopyAllReduce()
    with strategy.scope():
        model = unet_gauss(input_size=(height, width, 1), dropout_1=do1, dropout_2=do2)
        model.compile(loss=loss,
                      optimizer=optimizers.Adam(learning_rate=lr),
                      metrics=['accuracy', sensitivity, specificity, f1score, recall, mean_iou, dice_coef])

    # We create two instances with the same arguments
    train_datagen = ImageDataGenerator(horizontal_flip=True,
                                       height_shift_range=0.1,
                                       width_shift_range=0.1,
                                       rescale=1.0 / 255.0)
                                       # preprocessing_function=aug_func(aug_funcs))
    val_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

    # Provide the same seed and keyword arguments to the fit and flow methods
    seed = 1
    MASS_DATAGEN_TRN = join(BASE_DIR_SCOL_V1, 'train')
    MASS_DATAGEN_VAL = join(BASE_DIR_SCOL_V1, 'validation')


    train_image_generator = train_datagen.flow_from_directory(join(MASS_DATAGEN_TRN, 'images'),
                                                              target_size=(height, width),
                                                              color_mode='grayscale',
                                                              class_mode=None,
                                                              seed=seed,
                                                              batch_size=batch_size)
    train_mask_generator = train_datagen.flow_from_directory(join(MASS_DATAGEN_TRN, 'masks'),
                                                             target_size=(height, width),
                                                             color_mode='grayscale',
                                                             class_mode=None,
                                                             seed=seed,
                                                             batch_size=batch_size)
    val_image_generator = val_datagen.flow_from_directory(join(MASS_DATAGEN_VAL, 'images'),
                                                          target_size=(height, width),
                                                          color_mode='grayscale',
                                                          class_mode=None,
                                                          seed=seed,
                                                          batch_size=batch_size)
    val_mask_generator = val_datagen.flow_from_directory(join(MASS_DATAGEN_VAL, 'masks'),
                                                         target_size=(height, width),
                                                         color_mode='grayscale',
                                                         class_mode=None,
                                                         seed=seed,
                                                         batch_size=batch_size)

    # Combine generators into one which yields image and masks
    # print(image_generator[0])
    # print(mask_generator[0])
    train_generator = zip(train_image_generator, train_mask_generator)
    val_generator = zip(val_image_generator, val_mask_generator)

    print("Val steps: " + str(int(math.ceil(val_image_generator.n // batch_size))))
    model.fit_generator(train_generator,
                        steps_per_epoch=int(math.ceil(train_image_generator.n // batch_size)),
                        epochs=epoch,
                        verbose=1,
                        validation_data=val_generator,
                        validation_steps=int(math.ceil(val_image_generator.n // batch_size)),
                        max_queue_size=100,
                        workers=20,
                        # class_weight={0: 1, 1: 10},
                        callbacks=callbacks)

    del model
    tf.keras.backend.clear_session()

if __name__ == '__main__':
    BASE_DIR_SCOL_V1  = 'C:/Users/sungjoo/Desktop/Compression_V1/'
    # os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    # os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2"
    os.environ["CUDA_VISIBLE_DEVICES"] = '0'
    print("Train image length: " + str(len(os.listdir(join(BASE_DIR_SCOL_V1, 'train', 'images', '0')))))
    print("Train mask length: " + str(len(os.listdir(join(BASE_DIR_SCOL_V1, 'train', 'masks', '0')))))
    print("Val image length: " + str(len(os.listdir(join(BASE_DIR_SCOL_V1, 'validation', 'images', '0')))))
    print("Val mask length: " + str(len(os.listdir(join(BASE_DIR_SCOL_V1, 'validation', 'masks', '0')))))
    train_scoliosis(batch_size=2, loss_name='bce_logdice_loss', lr=0.0001, width=256, height=256, norm_func='gauss', do1=0.0, do2=0.0, cosine_lr=True, aug_funcs=[], spec_name='g-layer', epoch=50)
#     train_scoliosis(batch_size=16, loss_name='bce_logdice_loss', lr=0.0001, width=256, height=768, norm_func='gauss', do1=0.5, do2=0.5, cosine_lr=True, aug_funcs=[], spec_name='g-layer', epoch=600)
#     train_scoliosis(batch_size=16, loss_name='bce_logdice_loss', lr=0.0001, width=256, height=768, norm_func='gauss', do1=0.25, do2=0.25, cosine_lr=True, aug_funcs=[], spec_name='g-layer', epoch=600)
#     # train_scoliosis(batch_size=16, loss_name='bce_logdice_loss', lr=0.0001, image_length=512, norm_func='gauss', do1=0.75, do2=0.75, cosine_lr=True, aug_funcs=[], spec_name='g-layer', epoch=400)


Train image length: 1096
Train mask length: 1096
Val image length: 137
Val mask length: 137
Physical_devices
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Found 1096 images belonging to 1 classes.
Found 1096 images belonging to 1 classes.
Found 137 images belonging to 1 classes.
Found 137 images belonging to 1 classes.
Val steps: 68
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 548 steps, validate for 68 steps
Epoch 1/50
547/548 [============================>.] - ETA: 1s - loss: 2.7741 - accuracy: 0.7086 - sensitivity: 0.6447 - specificity: 0.7135 - f1score: 0.2255 - recall: 0.6841 - mean_iou: 0.1317 - dice_coef: 0.2257
Epoch 00001: val_loss improved from inf to 2.53088, saving model to C:/Users/sungjoo/Deepnoid/cf_weight/cf_train_no_bin_bckgnd_resized_unet_gaussresize_bce_logdice_loss_256256_bs-2_adam-cosine_lr-0.0001_do1_0.0_do2_0.0_aug_added_fctr_0

Epoch 20/50
547/548 [============================>.] - ETA: 3s - loss: 1.1588 - accuracy: 0.9734 - sensitivity: 0.9308 - specificity: 0.9835 - f1score: 0.8598 - recall: 0.9430 - mean_iou: 0.7538 - dice_coef: 0.8564
Epoch 00020: val_loss improved from 1.23832 to 1.19604, saving model to C:/Users/sungjoo/Deepnoid/cf_weight/cf_train_no_bin_bckgnd_resized_unet_gaussresize_bce_logdice_loss_256256_bs-2_adam-cosine_lr-0.0001_do1_0.0_do2_0.0_aug_added_fctr_0.5_g-layer-20.h5
548/548 [==============================] - 1992s 4s/step - loss: 1.1594 - accuracy: 0.9734 - sensitivity: 0.9308 - specificity: 0.9834 - f1score: 0.8595 - recall: 0.9430 - mean_iou: 0.7535 - dice_coef: 0.8561 - val_loss: 1.1960 - val_accuracy: 0.9795 - val_sensitivity: 0.9764 - val_specificity: 0.9802 - val_f1score: 0.8338 - val_recall: 0.9783 - val_mean_iou: 0.7187 - val_dice_coef: 0.8337
Epoch 21/50
547/548 [============================>.] - ETA: 3s - loss: 1.0630 - accuracy: 0.9736 - sensitivity: 0.9262 - specificity: 0.

Epoch 30/50
547/548 [============================>.] - ETA: 3s - loss: 0.4159 - accuracy: 0.9774 - sensitivity: 0.9100 - specificity: 0.9899 - f1score: 0.8893 - recall: 0.9211 - mean_iou: 0.7960 - dice_coef: 0.8841
Epoch 00030: val_loss improved from 0.46359 to 0.38842, saving model to C:/Users/sungjoo/Deepnoid/cf_weight/cf_train_no_bin_bckgnd_resized_unet_gaussresize_bce_logdice_loss_256256_bs-2_adam-cosine_lr-0.0001_do1_0.0_do2_0.0_aug_added_fctr_0.5_g-layer-30.h5
548/548 [==============================] - 1993s 4s/step - loss: 0.4158 - accuracy: 0.9774 - sensitivity: 0.9100 - specificity: 0.9899 - f1score: 0.8894 - recall: 0.9212 - mean_iou: 0.7961 - dice_coef: 0.8842 - val_loss: 0.3884 - val_accuracy: 0.9876 - val_sensitivity: 0.9535 - val_specificity: 0.9900 - val_f1score: 0.8938 - val_recall: 0.9553 - val_mean_iou: 0.8090 - val_dice_coef: 0.8937
Epoch 31/50
547/548 [============================>.] - ETA: 3s - loss: 0.3767 - accuracy: 0.9778 - sensitivity: 0.9073 - specificity: 0.

Epoch 40/50
547/548 [============================>.] - ETA: 3s - loss: 0.2273 - accuracy: 0.9778 - sensitivity: 0.8913 - specificity: 0.9922 - f1score: 0.8912 - recall: 0.8999 - mean_iou: 0.8021 - dice_coef: 0.8855
Epoch 00040: val_loss improved from 0.19493 to 0.18986, saving model to C:/Users/sungjoo/Deepnoid/cf_weight/cf_train_no_bin_bckgnd_resized_unet_gaussresize_bce_logdice_loss_256256_bs-2_adam-cosine_lr-0.0001_do1_0.0_do2_0.0_aug_added_fctr_0.5_g-layer-40.h5
548/548 [==============================] - 1997s 4s/step - loss: 0.2272 - accuracy: 0.9778 - sensitivity: 0.8913 - specificity: 0.9922 - f1score: 0.8912 - recall: 0.8999 - mean_iou: 0.8022 - dice_coef: 0.8856 - val_loss: 0.1899 - val_accuracy: 0.9885 - val_sensitivity: 0.9355 - val_specificity: 0.9919 - val_f1score: 0.8976 - val_recall: 0.9370 - val_mean_iou: 0.8153 - val_dice_coef: 0.8975
Epoch 41/50
547/548 [============================>.] - ETA: 3s - loss: 0.2304 - accuracy: 0.9770 - sensitivity: 0.8875 - specificity: 0.

In [56]:
import os
import cv2
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from tensorflow.keras.models import load_model

def set_gpus():
    # os.environ["CUDA_VISIBLE_DEVICES"] = gpu_nums
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
    # config.log_device_placement = True      # to log device placement (on which device the operation ran)
    sess = tf.Session(config=config)
    set_session(sess)


def load_model_to_gpus(model_path, gpu_nums, custom_objects=None):
    set_gpus()
    model = load_model(model_path, custom_objects=custom_objects)
    return model

custom_object_1 = {"dice_loss": dice_loss,
                   "sensitivity": sensitivity,
                   "specificity": specificity,
                   "f1score": f1score,
                   "recall": recall,
                   "mean_iou": mean_iou,
                   "bce_logdice_loss": bce_logdice_loss,
                   "dice_coef": dice_coef,
                   "precision": precision}

if __name__ == '__main__':
    os.environ["CUDA_VISIBLE_DEVICES"] = '0'
    sc_1_model = load_model('C:/Users/sungjoo/Deepnoid/cf_weight/cf_train_no_bin_bckgnd_resized_unet_gaussresize_bce_logdice_loss_256256_bs-2_adam-cosine_lr-0.0001_do1_0.0_do2_0.0_aug_added_fctr_0.5_g-layer-48.h5'
                            ,custom_objects=custom_object_1)
#     sc_2_model = load_model('/data/hira2020/scoliosis/hyoon/weight_dir/'
#                             'scol_train_no_bin_resized_unet_gaussresize_bce_logdice_loss_768_bs-16_adam-cosine_lr-5e-05'
#                             '_do1_0.75_do2_0.75_aug_added_fctr_0.5_2nd-seg-600.h5',
#                             custom_objects=custom_object_1)
    # sc_3_model = load_model('/data/hira2020/scoliosis/hyoon/weight_dir/scol_patch_train_unet_gaussbce_logdice_loss_512_bs-16_adam-cosine_lr-5e-05_do1_0.0_do2_0.0_3rd-seg-36.h5',
    #                         custom_objects=custom_object_1)

    orig_dir = 'C:/Users/sungjoo/Desktop/compression_V1/test/images/0/'
    orig_files = os.listdir(orig_dir)
#     label_dir = '/data/hira2020/scoliosis/hyoon/scoliosis_v1/png_label/'
#     label_files = os.listdir(label_dir)
    # label_path = '/data/hira2020/scoliosis/hyoon/scoliosis_v1/png_label/'

#     cropped_dir = '/data/hira2020/scoliosis/hyoon/cropped_image/'
    for i, of in enumerate(orig_files):
        print("#"*30)
        print(str(i+1)+"/"+str(len(orig_files)) + ": " + of)
        orig_file_path = os.path.join(orig_dir, of)
        orig_img = cv2.imread(orig_file_path, 0)
        print(orig_img.shape)
        # 1st model execution
        prep_img_1= sc_fst_mdl_pre_procssing(orig_img)
        prep_img_1 = np.asarray(prep_img_1)
        sc_1_pred = sc_1_model.predict(prep_img_1.reshape(256,256,1))
        sc_1_pred = sc_1_pred*255
        cv2.imwrite('C:/Users/sungjoo/Deepnoid/pred_file/' + of[:-4]+'_pred.png', sc_1_pred)
#         cropped_image = sc_fst_mdl_post_processing(orig_img, sc_1_pred, crop_coords, orig_img.shape, of)

#         # 2nd model execution
#         prep_img_2 = sc_snd_mdl_pre_procssing(cropped_image)
#         # cv2.imwrite('/data/hira2020/scoliosis/hyoon/restored_image/' + of[:-4] + '_2nd_prep.png', prep_img_2)
#         print(prep_img_2.shape)
#         sc_2_pred = sc_2_model.predict(prep_img_2.reshape((1, 768, 256, 1))).reshape((768, 256, 1))
#         cropped_images = sc_snd_mdl_post_processing(cropped_image, sc_2_pred, cropped_image.shape, of)

#         prep_label_2 = sc_snd_mdl_pre_procssing(orig_label)
#         # print(prep_img_2.shape)
#         # sc_2_pred = sc_2_model.predict(prep_img_2.reshape((1, 768, 256, 1)).reshape((768, 256, 1))
#         _, cropped_labels = sc_snd_mdl_post_processing(cropped_image, prep_label_2, cropped_image.shape, lf)
#         cv2.imwrite('/data/hira2020/scoliosis/hyoon/sample_label_res/' + lf[:-4] + '_2nd_prep.png', cropped_labels)

        # #3rd model execution
        # for i in os.listdir(cropped_dir):
        #     img = os.path.join(cropped_dir, i)
        #     cropped = cv2.imread(img, 0)
        #     print(cropped.shape)
        #
        #     sc_3_pred = sc_3_model.predict(sc_fst_mdl_pre_procssing(cropped)[0][None,:,:,None])
        #     sc_3_pred = sc_3_pred[0,:,:,0]
        #     sc_3_pred=sc_3_pred*255
        #     cropped_images = sc_snd_mdl_post_processing(cropped, sc_3_pred, cropped.shape, i)
        #     # cv2.imwrite('/data/hira2020/scoliosis/hyoon/)
        #     # cropped_images = sc_snd_mdl_post_processing(cropped, sc_3_pred, cropped.shape, of)


ResourceExhaustedError: OOM when allocating tensor with shape[3,3,128,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Mul] name: conv2d_4_20/kernel/Initializer/truncated_normal/mul/

In [24]:
import math
import cv2
import gc
import numpy as np

# import src.processing.pre_processing.classification.deepphi_crc_modules.config as c
# from utility.image import shape, crop


def resizing_image(image, height_and_width, color='white'):

    try:
        if not isinstance(image, np.ndarray):
            image = np.asarray(image)

        if image.shape == (height_and_width, height_and_width):
            return image, (0, 0, height_and_width, height_and_width)
        elif (image.shape[0] < height_and_width) and (image.shape[1] < height_and_width):
            size = image.shape[:2]
            delta_w = height_and_width - size[1]
            delta_h = height_and_width - size[0]
            top, bottom = delta_h // 2, delta_h - (delta_h // 2)
            left, right = delta_w // 2, delta_w - (delta_w // 2)
            # print("Resizing Log: ")
            # print(left, top, right, bottom)
            # print("delta_w: " + str(delta_w))
            # print("delta_h: " + str(delta_h))
            return cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=0), (left, top, height_and_width-right, height_and_width-bottom)
        else:
            old_size = image.shape[:2]
            ratio = float(height_and_width) / max(old_size)
            new_size = tuple([int(x * ratio) for x in old_size])
            im = cv2.resize(image, (new_size[1], new_size[0]))
            delta_w = height_and_width - new_size[1]
            delta_h = height_and_width - new_size[0]
            top, bottom = delta_h // 2, delta_h - (delta_h // 2)
            left, right = delta_w // 2, delta_w - (delta_w // 2)
            # print("Resizing Log: ")
            # print(left, top, right, bottom)
            # print("delta_w: " + str(delta_w))
            # print("delta_h: " + str(delta_h))
            return cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT, value=0), (left, top, height_and_width-right, height_and_width-bottom)
    except Exception as error:
        raise Exception("Exception occurred while resizing the image: " + str(error))


def resizing_image_wh(image, width, height):

    try:
        if not isinstance(image, np.ndarray):
            image = np.asarray(image)
        print(image.shape)
        im = cv2.resize(image, (width, height))
        return im
    except Exception as error:
        raise Exception("Exception occurred while resizing the image: " + str(error))


In [37]:
import os
import cv2
import numpy as np
# from ai_inference.playground.resize_or_extract import resizing_image, resizing_image_wh


def sc_fst_mdl_pre_procssing(image):
#     resized_image, crop_coords = resizing_image(image, 256)
    normalized_image = cv2.normalize(image, 0, 255, norm_type=cv2.NORM_MINMAX)
    normalized_image = normalized_image * (1./255.)
    return normalized_image


if __name__ == '__main__':
    pass
